In [16]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

In [17]:
#!git clone https://github.com/SIDN-IAP/global-model-repr.git datasource
!pip install transformers==2.1
!pip install spacy ftfy==4.4.3
!python -m spacy download en

     ---------------------------------------- 12.8/12.8 MB 8.1 MB/s eta 0:00:00
[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use
the full pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [18]:
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
import sys

sys.path.append('./datasource')

if torch.cuda.is_available():
    device = torch.device('cpu')
else:
    print("Change runtime type to include a GPU.")  
    device = torch.device('cpu')
print("device:", device)

device: cpu


In [19]:
import torch
import os
import numpy as np
from transformers import GPT2Tokenizer, GPT2Model, BertTokenizer, BertModel

DATA_DIR = "data"
UD_EN_PREF = "en-ud-"

def get_model_and_tokenizer(model_name, device, random_weights=False):

    model_name = model_name

    if model_name.startswith('gpt2'):
        model = GPT2Model.from_pretrained(model_name, output_hidden_states=True).to(device)
        tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        sep = 'Ġ'
        sizes = {"gpt2": 768, "gpt2-medium": 1024, "gpt2-large": 1280, "gpt2-xl": 1600}
        emb_dim = sizes[model_name]
    elif model_name.startswith('bert'):
        model = BertModel.from_pretrained(model_name, output_hidden_states=True).to(device)
        tokenizer = BertTokenizer.from_pretrained(model_name)
        sep = '##'
        emb_dim = 1024 if "large" in model_name else 768
    else:
        print('Unrecognized model name:', model_name)
        sys.exit()

    if random_weights:
        print('Randomizing weights')
        model.init_weights()

    return model, tokenizer, sep, emb_dim

In [20]:
def get_sentence_repr(sentence, model, tokenizer, sep, model_name, device):
    """
    Get representations for one sentence
    """

    with torch.no_grad():
        ids = tokenizer.encode(sentence)
        input_ids = torch.tensor([ids]).to(device)
        # Hugging Face format: list of torch.FloatTensor of shape (batch_size, sequence_length, hidden_size) (hidden_states at output of each layer plus initial embedding outputs)
        all_hidden_states = model(input_ids)[-1]
        # convert to format required for contexteval: numpy array of shape (num_layers, sequence_length, representation_dim)
        all_hidden_states = [hidden_states[0].cpu().numpy() for hidden_states in all_hidden_states]
        all_hidden_states = np.array(all_hidden_states)

    #For each word, take the representation of its last sub-word
    segmented_tokens = tokenizer.convert_ids_to_tokens(ids)
    assert len(segmented_tokens) == all_hidden_states.shape[1], 'incompatible tokens and states'
    mask = np.full(len(segmented_tokens), False)

    if model_name.startswith('gpt2'):
        # if next token is a new word, take current token's representation
        #print(segmented_tokens)
        for i in range(len(segmented_tokens)-1):
            if segmented_tokens[i+1].startswith(sep):
                #print(i)
                mask[i] = True
        # always take the last token representation for the last word
        mask[-1] = True
    # example: ['jim</w>', 'henson</w>', 'was</w>', 'a</w>', 'pup', 'pe', 'teer</w>']
    elif model_name.startswith('bert'):
        # if next token is not a continuation, take current token's representation
        for i in range(len(segmented_tokens)-1):
            if not segmented_tokens[i+1].startswith(sep):
                mask[i] = True
        mask[-1] = True
    else:
        print('Unrecognized model name:', model_name)
        sys.exit()

    all_hidden_states = all_hidden_states[:, mask]
    # all_hidden_states = torch.tensor(all_hidden_states).to(device)

    return all_hidden_states

In [21]:
def get_pos_data(probing_dir, frac=1.0, device='cpu'):

    return get_data("pos", probing_dir=probing_dir, frac=frac, device=device)

In [22]:
def get_data(data_type, probing_dir, data_pref=UD_EN_PREF, frac=1.0, device='cpu'):

    with open(os.path.join(probing_dir, DATA_DIR, data_pref + "train.txt"), encoding="utf8") as f:
        train_sentences = [line.strip().split() for line in f.readlines()]
    with open(os.path.join(probing_dir, DATA_DIR, data_pref + "test.txt"), encoding="utf8") as f:
        test_sentences = [line.strip().split() for line in f.readlines()]
    with open(os.path.join(probing_dir, DATA_DIR, data_pref + "dev.txt"), encoding="utf8") as f:
        dev_sentences = [line.strip().split() for line in f.readlines()]

    with open(os.path.join(probing_dir, DATA_DIR, data_pref + "train." + data_type), encoding="utf8") as f:
        train_labels = [line.strip().split() for line in f.readlines()]
    with open(os.path.join(probing_dir, DATA_DIR, data_pref + "test." + data_type), encoding="utf8") as f:
        test_labels = [line.strip().split() for line in f.readlines()]
    with open(os.path.join(probing_dir, DATA_DIR, data_pref + "dev." + data_type), encoding="utf8") as f:
        dev_labels = [line.strip().split() for line in f.readlines()]

    # take a fraction of the data
    train_sentences = train_sentences[:round(len(train_sentences)*frac)]
    test_sentences = train_sentences[:round(len(test_sentences)*frac)]
    dev_sentences = train_sentences[:round(len(dev_sentences)*frac)]
    train_labels = train_labels[:round(len(train_labels)*frac)]
    test_labels = train_labels[:round(len(test_labels)*frac)]
    dev_labels = train_labels[:round(len(dev_labels)*frac)]

    unique_labels = list(set.union(*[set(l) for l in train_labels + test_labels + dev_labels]))
    label2index = dict()
    for label in unique_labels:
        label2index[label] = label2index.get(label, len(label2index))

    train_labels = [[label2index[l] for l in labels] for labels in train_labels]
    test_labels = [[label2index[l] for l in labels] for labels in test_labels]
    dev_labels = [[label2index[l] for l in labels] for labels in dev_labels]


    return train_sentences, train_labels, test_sentences, test_labels, dev_sentences, dev_labels, label2index

In [23]:
train_sentences, train_labels, test_sentences, test_labels, _, _, label2index = get_pos_data("./datasource/probing/", frac=0.1)
# train_sentences, train_labels, test_sentences, test_labels, _, _, label2index = get_pos_data("../probing", frac=0.1)
num_labels = len(label2index)
print("Training sentences:", len(train_sentences), "Test sentences:", len(test_sentences))
print("Unique labels:", num_labels)

Training sentences: 1254 Test sentences: 208
Unique labels: 17


In [24]:
class Classifier(torch.nn.Module):
    
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, input):
        output = self.linear(input)
        return output
    

class NonlinearClassifier(torch.nn.Module):
    
    def __init__(self, input_dim, output_dim):
        super(NonlinearClassifier, self).__init__()
        
        self.input2hidden = torch.nn.Linear(input_dim, input_dim)
        self.hidden2output = torch.nn.Linear(input_dim, output_dim)
        self.relu = torch.nn.ReLU()
        
    def forward(self, input):
        hidden = self.relu(self.input2hidden(input))
        output = self.hidden2output(hidden)
        return output
    
    
def build_classifier(emb_dim, num_labels, device='cpu'):

    classifier = Classifier(emb_dim, num_labels).to(device)
    criterion = torch.nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(classifier.parameters())

    return classifier, criterion, optimizer


def build_nonlinear_classifier(emb_dim, num_labels, device='cpu'):

    classifier = NonlinearClassifier(emb_dim, num_labels).to(device)
    criterion = torch.nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(classifier.parameters())

    return classifier, criterion, optimizer


model_name = 'bert-base-cased'
# get model and tokenizer from Transformers
model, tokenizer, sep, emb_dim = get_model_and_tokenizer(model_name, device)
# build classifier
classifier, criterion, optimizer = build_classifier(emb_dim, num_labels, device)

In [25]:
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [26]:
print(classifier)

Classifier(
  (linear): Linear(in_features=768, out_features=17, bias=True)
)


In [27]:
def train(num_epochs, train_representations, train_labels, 
          model, tokenizer, sep, model_name, device, 
          classifier, criterion, optimizer, batch_size=32):
    
    num_total = train_representations.shape[0] 
    for i in range(num_epochs):
        total_loss = 0.
        num_correct = 0.
        for batch in range(0, num_total, batch_size):
            batch_repr = train_representations[batch: batch+batch_size]
            batch_labels = train_labels[batch: batch+batch_size]

            optimizer.zero_grad()
            
            out = classifier(batch_repr)
            pred = out.max(1)[1]
            num_correct += pred.long().eq(batch_labels.long()).cpu().sum().item()
            loss = criterion(out, batch_labels.type(torch.LongTensor))
            total_loss += loss.item()

            loss.backward()
            optimizer.step()
#         print('Training epoch: {}, loss: {}, accuracy: {}'.format(i, total_loss/num_total, num_correct/num_total))
    return total_loss/num_total, num_correct/num_total

In [28]:
def evaluate(test_representations, test_labels, 
             model, tokenizer, sep, model_name, device, 
             classifier, criterion, batch_size=32):
    
    num_correct = 0.
    num_total = test_representations.shape[0]
    total_loss = 0.
    with torch.no_grad():
        for batch in range(0, num_total, batch_size):
            batch_repr = test_representations[batch: batch+batch_size]
            batch_labels = test_labels[batch: batch+batch_size]
            
            out = classifier(batch_repr)
            pred = out.max(1)[1]
            num_correct += pred.long().eq(batch_labels.long()).cpu().sum().item()
            total_loss += criterion(out, batch_labels.type(torch.LongTensor))

#     print('Testing loss: {}, accuracy: {}'.format(total_loss/num_total, num_correct/num_total))
    return total_loss/num_total, num_correct/num_total

In [29]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [30]:
# top-level list: sentences, second-level lists: layers, third-level tensors of num_words x representation_dim
train_sentence_representations = [get_sentence_repr(sentence, model, tokenizer, sep, model_name, device) 
                                  for sentence in train_sentences]
test_sentence_representations = [get_sentence_repr(sentence, model, tokenizer, sep, model_name, device) 
                                  for sentence in test_sentences]

# top-level list: layers, second-level lists: sentences
train_sentence_representations = [list(l) for l in zip(*train_sentence_representations)]
test_sentence_representations = [list(l) for l in zip(*test_sentence_representations)]                           

# concatenate all word represenations
train_representations_all = [torch.tensor(np.concatenate(train_layer_representations, 0)).to(device) for train_layer_representations in train_sentence_representations]
test_representations_all = [torch.tensor(np.concatenate(test_layer_representations, 0)).to(device) for test_layer_representations in test_sentence_representations]
# concatenate all labels
train_labels_all = torch.tensor(np.concatenate(train_labels, 0)).to(device)
test_labels_all = torch.tensor(np.concatenate(test_labels, 0)).to(device)

In [31]:
print(len(train_sentences))

1254


In [32]:
# Take final layer representations
train_representations = train_representations_all[-1]
test_representations = test_representations_all[-1]

# train
train_loss, train_accuracy = train(10, train_representations, train_labels_all, 
          model, tokenizer, sep, model_name, device, 
          classifier, criterion, optimizer)
# test
test_loss, test_accuracy = evaluate(test_representations, test_labels_all, 
         model, tokenizer, sep, model_name, device, 
         classifier, criterion)
print("Train accuracy: {}, Test accuracy: {}".format(train_accuracy, test_accuracy))

Train accuracy: 0.9266106027488609, Test accuracy: 0.9172932330827067


In [33]:
num_layers = len(train_representations_all)
train_accs, test_accs = [], []
for l in range(num_layers):
    # build new classifier for every layer experiment
    classifier, criterion, optimizer = build_classifier(emb_dim, num_labels, device)
    # get layer representation 
    train_representations = train_representations_all[l]
    test_representations = test_representations_all[l]
    
    # train
    train_loss, train_accuracy = train(2, train_representations, train_labels_all, 
          model, tokenizer, sep, model_name, device, 
          classifier, criterion, optimizer)
    train_accs.append(train_accuracy)
    # test
    test_loss, test_accuracy = evaluate(test_representations, test_labels_all, 
         model, tokenizer, sep, model_name, device, 
         classifier, criterion)
    test_accs.append(test_accuracy)
    print("layer: {}, train accuracy: {}, test accuracy: {}".format(l, train_accuracy, test_accuracy))

layer: 0, train accuracy: 0.8647427110732412, test accuracy: 0.8469703670942061
layer: 1, train accuracy: 0.8850072240951358, test accuracy: 0.8759398496240601
layer: 2, train accuracy: 0.9243878042455451, test accuracy: 0.9296771340114993
layer: 3, train accuracy: 0.9296113807283369, test accuracy: 0.9283502874834144
layer: 4, train accuracy: 0.9333901381839736, test accuracy: 0.930561698363556
layer: 5, train accuracy: 0.9325751120660912, test accuracy: 0.9301194161875277
layer: 6, train accuracy: 0.9322416922905938, test accuracy: 0.9296771340114993
layer: 7, train accuracy: 0.931648946023043, test accuracy: 0.9256965944272446
layer: 8, train accuracy: 0.9244989441707109, test accuracy: 0.9201680672268907
layer: 9, train accuracy: 0.9147927240395658, test accuracy: 0.9117647058823529
layer: 10, train accuracy: 0.9038269180898751, test accuracy: 0.9035824856258293
layer: 11, train accuracy: 0.8934538584077354, test accuracy: 0.8971693940734189
layer: 12, train accuracy: 0.86515022413

In [34]:
import random

vocabulary = set(
    word
      for sentence in (train_sentences + test_sentences)
      for word in sentence
)
# all_labels = sum((x.tolist() for x in train_labels), [])
all_labels = train_labels_all.tolist()
control_map = {word: random.choice(all_labels) for word in vocabulary}

control_train_labels = [torch.tensor([control_map[word] for word in sentence]) for sentence in train_sentences]
control_test_labels = [torch.tensor([control_map[word] for word in sentence]) for sentence in test_sentences]
control_train_labels = torch.tensor(np.concatenate(control_train_labels, 0)).to(device)
control_test_labels = torch.tensor(np.concatenate(control_test_labels, 0)).to(device)

In [35]:
num_layers = len(train_representations_all)
control_train_accs, control_test_accs = [], [] 
for l in range(num_layers):
    classifier, criterion, optimizer = build_classifier(emb_dim, num_labels, device)
    # get layer representation 
    train_representations = train_representations_all[l]
    test_representations = test_representations_all[l]
    
    # train
    train_loss, train_accuracy = train(2, train_representations, control_train_labels, 
          model, tokenizer, sep, model_name, device, 
          classifier, criterion, optimizer)
    control_train_accs.append(train_accuracy)
    # test
    test_loss, test_accuracy = evaluate(test_representations, control_test_labels, 
         model, tokenizer, sep, model_name, device, 
         classifier, criterion)
    control_test_accs.append(test_accuracy)    
    print("layer: {}, train accuracy: {}, test accuracy: {}".format(l, train_accuracy, test_accuracy))

layer: 0, train accuracy: 0.7168154706775831, test accuracy: 0.7293233082706767
layer: 1, train accuracy: 0.7085911162153151, test accuracy: 0.7204776647501105
layer: 2, train accuracy: 0.6924758270662764, test accuracy: 0.7041132242370632
layer: 3, train accuracy: 0.675730745007965, test accuracy: 0.6943830163644406
layer: 4, train accuracy: 0.6617271144370763, test accuracy: 0.6786819991154357
layer: 5, train accuracy: 0.6435742599933316, test accuracy: 0.66187527642636
layer: 6, train accuracy: 0.626940317860186, test accuracy: 0.6410880141530296
layer: 7, train accuracy: 0.6030452339495425, test accuracy: 0.6076957098628926
layer: 8, train accuracy: 0.5792612899640648, test accuracy: 0.5888987173816895
layer: 9, train accuracy: 0.5579965176156781, test accuracy: 0.5667846085802742
layer: 10, train accuracy: 0.5349905531063609, test accuracy: 0.5486510393631137
layer: 11, train accuracy: 0.5252472863334938, test accuracy: 0.5367094206103494
layer: 12, train accuracy: 0.4962397658652

In [36]:
for l in range(num_layers):
    print("layer: {}, test selectivity: {}".format(l, test_accs[l] - control_test_accs[l]))

layer: 0, test selectivity: 0.11764705882352944
layer: 1, test selectivity: 0.15546218487394958
layer: 2, test selectivity: 0.22556390977443608
layer: 3, test selectivity: 0.23396727111897386
layer: 4, test selectivity: 0.25187969924812026
layer: 5, test selectivity: 0.2682441397611677
layer: 6, test selectivity: 0.2885891198584697
layer: 7, test selectivity: 0.31800088456435205
layer: 8, test selectivity: 0.3312693498452012
layer: 9, test selectivity: 0.3449800973020787
layer: 10, test selectivity: 0.35493144626271567
layer: 11, test selectivity: 0.36045997346306946
layer: 12, test selectivity: 0.36731534719150816
